In [ ]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [ ]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=1,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([9]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

In [ ]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
)

In [ ]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [ ]:
order_records_df